In [29]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [30]:
#data sugmentation
# Preprocessing the Training set
datagen = ImageDataGenerator(rescale=1./255,validation_split=0.22)
training_set = datagen.flow_from_directory('train',
                                            target_size=(224,224),
                                            color_mode="rgb",
                                            subset="training",
                                            class_mode="binary",
                                            batch_size=32,
                                            shuffle=True)

Found 198 images belonging to 2 classes.


In [31]:
valid_generator=datagen.flow_from_directory( directory="train",
                                                target_size=(224,224),
                                                color_mode="rgb",
                                                subset="validation",
                                                class_mode="binary",
                                                batch_size=32,
                                                shuffle=True)

Found 55 images belonging to 2 classes.


In [32]:
training_set.class_indices

{'class_no': 0, 'class_yes': 1}

In [33]:
import tensorflow_hub as hub 

In [34]:
BATCH_SIZE = 5
IMG_SHAPE = 224 # match image dimension to mobile net input keep smae

In [35]:
# getting MobileNet
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobile_net = hub.KerasLayer(URL, input_shape=(IMG_SHAPE, IMG_SHAPE, 3))

In [36]:
mobile_net.trainable = False

In [37]:
model = tf.keras.models.Sequential([
    mobile_net,
    ### ann layer
    tf.keras.layers.Dense(1, activation='sigmoid') #[0, 1] or [1, 0]
    ])

In [38]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
history = model.fit(
    training_set,
    epochs=10,
    validation_data=valid_generator
    )

Epoch 1/10
7/7 [==============================] - 32s 1s/step - loss: 0.7839 - accuracy: 0.5152 - val_loss: 0.7415 - val_accuracy: 0.5818
Epoch 2/10
7/7 [==============================] - 5s 749ms/step - loss: 0.6634 - accuracy: 0.6111 - val_loss: 0.6595 - val_accuracy: 0.6182
Epoch 3/10
7/7 [==============================] - 4s 522ms/step - loss: 0.5831 - accuracy: 0.7222 - val_loss: 0.5993 - val_accuracy: 0.6182
Epoch 4/10
7/7 [==============================] - 5s 658ms/step - loss: 0.5159 - accuracy: 0.7929 - val_loss: 0.5599 - val_accuracy: 0.6727
Epoch 5/10
7/7 [==============================] - 4s 519ms/step - loss: 0.4745 - accuracy: 0.8030 - val_loss: 0.5134 - val_accuracy: 0.7273
Epoch 6/10
7/7 [==============================] - 4s 539ms/step - loss: 0.4349 - accuracy: 0.8485 - val_loss: 0.4859 - val_accuracy: 0.7455
Epoch 7/10
7/7 [==============================] - 4s 601ms/step - loss: 0.4076 - accuracy: 0.8586 - val_loss: 0.4586 - val_accuracy: 0.7455
Epoch 8/10
7/7 [======

In [40]:
# Part 4 - Making a single prediction test
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('9 no.jpg', target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
result

array([[0.39445272]], dtype=float32)

In [41]:
if result[0]<=0.5:
    print("No BrainTumor")
else:
    print("BrainTumor")

No BrainTumor


In [42]:
#save model
from tensorflow.keras.models import load_model
model.save('brain_tumor.h5')